## Chapter 21: Advanced Function Features of Julia

This chapter covers a bit more on functions in julia. These feature allow the ability to write code that easier to use, read and debug. We will perform error checking for arguments to ensure that only valid arguments are considered. Additionally, to make functions more robust, we’ll use option arguments and keyword arguments.

#### 21.1: Testing Arguments

Recall the recursive factorial function:

In [1]:
fact(n::Integer) = n==0 ? 1 : n*fact(n-1)

fact (generic function with 1 method)

If we put in a negative number, then

In [2]:
fact(-5)

StackOverflowError: StackOverflowError:

This occurs because fact(-5) would return -5 times fact(-6) which would return -5 times -6 times fact(-7), etc.  This will never stop. 

To prevent this, we will check if the argument n is greater than or equal to 0. The first line of the function, evaluates `n>=0`. If this is true, it skips the rest of the line.  If not, it throws an error. 

In [4]:
function fact(n::Integer)
  n>=0 || throw(ArgumentError("The argument must be a nonnegative integer."))
  n==0 ? 1 : n*fact(n-1)
end

fact (generic function with 1 method)

In [5]:
fact(-5)

ArgumentError: ArgumentError: The argument must be a nonnegative integer.

### 21.2: Optional arguments

Recall Newton's method that we saw earlier:

In [6]:
using ForwardDiff
function newton(f::Function, x0::Number)
  local dx = f(x0)/ForwardDiff.derivative(f,x0)
  local steps = 0
  while abs(dx)>1e-6 && steps<10
    x0 -= dx
    dx = f(x0)/ForwardDiff.derivative(f,x0)
    steps += 1
  end
  x0
end

newton (generic function with 1 method)

In [7]:
newton(x->x^2-5,1)

2.2360688956433634

Note: we haven't used the `Root` datatype developed in Chapter 12 for simplicity.  An exercise would be to combine this.

What if we want the root tolerance (1e-6) to be better or to run more steps than 10.  These are hard coded parameters that would have to be updated in the newton function.  A much better way to do this is to add these to the argument list, but with default values.  Julia calls these optional arguments.  Consider this rewrite:

In [8]:
function newton(f::Function, x0::Number, tol=1e-6, max_steps=10)
  local dx = f(x0)/ForwardDiff.derivative(f,x0)
  local steps = 0
  while abs(dx) > tol && steps < max_steps
    x0 -= dx
    dx = f(x0)/ForwardDiff.derivative(f,x0)
    steps += 1
  end
  x0
end

newton (generic function with 3 methods)

In [9]:
newton(x->x^2-5,1)

2.2360688956433634

In [10]:
methods(newton)

# 3 methods for generic function "newton" from Main:
 [1] newton(f::Function, x0::Number, tol, max_steps)
     @ ~/code/sci-comp-notebooks/notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X21sZmlsZQ==.jl:1
 [2] newton(f::Function, x0::Number, tol)
     @ ~/code/sci-comp-notebooks/notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X21sZmlsZQ==.jl:1
 [3] newton(f::Function, x0::Number)
     @ ~/code/sci-comp-notebooks/notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X21sZmlsZQ==.jl:1

First, notice that there are 3 methods now. What are the signatures of newton for this?

Now, notice that we can change the tolerance with:

In [11]:
newton(x->x^2-5,1,1e-12)

2.236067977499978

But a problem is that to change the maximum number of steps, we need to change both:

In [14]:
newton(x->x^2-5,1,1e-12,20)

2.236067977499978

We don't need to set the argument types for optional arguments.  Julia will figure out the type by the number that you put in.  The example above, `tol` will be a Float and `max_steps` will be an Integer. 

We should also check these new arguments have proper types.  Add lines at the top of the following function to handle this argument check:

In [15]:
function newton(f::Function, x0::Number,tol=1e-6,max_steps=10)
  tol > 0 || throw(ArgumentError("The value of tol must be positive."))
  max_steps > 0 || throw(ArgumentError("The value of max_steps must be positive."))
  local dx = f(x0)/ForwardDiff.derivative(f,x0)
  local steps = 0
  while abs(dx)>tol && steps<max_steps
    x0 -= dx
    dx = f(x0)/ForwardDiff.derivative(f,x0)
    steps += 1
  end
  x0
end

newton (generic function with 3 methods)

In [16]:
newton(x->x^2-5,1,-0.01)

ArgumentError: ArgumentError: The value of tol must be positive.

### 21.3: Handling Special Cases

In [17]:
newton(x->x^2+2,1)

0.5353752385394379

We looked at this earlier, but recall that if we reach too many steps.  In this case, the number of steps is `max_steps`, then we want to throw an error.  We can use the same notation with the || shortcut to do this. 

In [18]:
using ForwardDiff
function newton(f::Function, x0::Number,tol=1e-6,max_steps=10)
  tol > 0 || throw(ArgumentError("The parameter tol must be positive"))
  max_steps > 0 || throw(ArgumentError("The parameter max_steps must be positive"))
  local dx = f(x0)/ForwardDiff.derivative(f,x0)
  local steps = 0
  while abs(dx)>tol && steps<max_steps
    x0 -= dx
    dx = f(x0)/ForwardDiff.derivative(f,x0)
    steps += 1
  end
  local error = "The maximum number of steps: $max_steps was reached without convergence"
  steps < max_steps || throw(ErrorException(error))
  x0
end

newton (generic function with 3 methods)

In [20]:
newton(x->x^2+2,1, 1e-10,100)

ErrorException: The maximum number of steps: 100 was reached without convergence

### 21.4: Keyword Arguments

Instead of optional arguments (especially if there are more than one), it often better to use keyword arugments in which you can change any of the arguments in any order. To do this, we need to use the keyword when calling the function.  To change from optional arguments to keyword arguments, separate the regular arguments from the keyword ones with a semicolon (;)

In [22]:
function newton(f::Function, x0::Number; tol=1e-6, max_steps=10)
  tol > 0 || throw(ArgumentError("The parameter tol much be positive"))
  max_steps > 0 || throw(ArgumentError("The parameter max_steps much be positive"))
  local x1 = x0
  local dx = f(x0)/ForwardDiff.derivative(f,x0)
  local steps = 0
  while abs(dx)>tol && steps<max_steps
    x0 -= dx
    dx = f(x0)/ForwardDiff.derivative(f,x0)
    steps += 1
  end
  local error = "The maximum number of steps: $max_steps was reached without convergence"
  steps < max_steps || throw(ErrorException(error))
  x0
end

newton (generic function with 3 methods)

In [23]:
newton(x->x^2-5,1,max_steps=5)

2.2360688956433634

In [24]:
newton(x->x^2-5,1,tol=1e-3)

2.2360688956433634

### 21.5: Parametric Methods

In Chapter 4, we saw multiple dispatch for julia.  Let's look at another example of this.  This will find the maximum value with different arguments put in. 

If we have two values we can do:

In [25]:
findMax(x::Real,y::Real) = x > y ? x : y

findMax (generic function with 1 method)

In [26]:
findMax(3,4)

4

As as we saw earlier, if we want any number of arguments, we could do:

In [29]:
function findMax(nums::Real...)
  local max = -Inf
  for num in nums
    if num > max
      max = num
    end
  end
  max
end

findMax (generic function with 2 methods)

In [30]:
findMax(1,6,2,3,-9,11//2,5.6)

6

It makes sense to also write a `findMax` function that takes an array of values:

In [31]:
function findMax(arr::Vector{Real})
  local max = -Inf
  for num in arr
    if num > max
      max = num
    end
  end
  max
end

findMax (generic function with 3 methods)

Let's say we want to find the max of `[-3,2,4,7,5]`.  It seems like the following should work:

In [32]:
findMax([-3,2,4,7,5])

MethodError: MethodError: no method matching findMax(::Vector{Int64})
The function `findMax` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  findMax(!Matched::Vector{Real})
   @ Main ~/code/sci-comp-notebooks/notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X61sZmlsZQ==.jl:1
  findMax(!Matched::Real, !Matched::Real)
   @ Main ~/code/sci-comp-notebooks/notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X52sZmlsZQ==.jl:1
  findMax(!Matched::Real...)
   @ Main ~/code/sci-comp-notebooks/notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X55sZmlsZQ==.jl:1


This happened because an array of type `Vector{Int64}` isn't matched by the argument type `Vector{Real}`.  Instead we do a parametric function, which is similar to the parametric type that we saw with `Polynomial` in Chapter 12. 

In [33]:
Vector{Int64} <: Vector{Real}

false

However, notice that the error says that it can't find a `findMax` function with argument `Vector{Int64}`.  

We could create functions to handle arrays of integers, floats, and other types that we need, but julia has what is called parametric methods that handles all of these together:

In [34]:
function findMax(arr::Vector{T}) where {T <: Real}
  local max = -Inf
  for num in arr
    if num > max
      max = num
    end
  end
  max
end

findMax (generic function with 4 methods)

which can now handle any array with any subtype of `Real`.

In [35]:
findMax([-3,2,4,7,5])

7

In [36]:
typeof([-3,2,4,7,5])

Vector{Int64} (alias for Array{Int64, 1})

In [37]:
findMax([pi,1.0,-6.3, 6/8])

3.141592653589793

In [38]:
findMax([big(3),big(9)])

9

In [39]:
findMax([6//7, -99//8, 4//5])

6//7

### 21.6: Function Arguments

Function arguments follow what is called "pass-by-sharing", which means that values are not copied when they are passed to functions.  Instead, the argument is a local variable that is passed the value when the function is called. 

In [42]:
x=11

11

In [43]:
g(x::Real) = x^2

g (generic function with 1 method)

In [44]:
g(3)

9

In [45]:
x

11

When calling `g(3)`, the variable `x` within the function becomes a local variable and the value 3 is assigned to this.

In [46]:
function double(x::Real)
  @show "inside double"
  @show x
  x *= 2
  @show x
  x
end

double (generic function with 1 method)

If we have this function, let's define a global variable `x` and give it a value, then call `double`

In [47]:
x=11
@show double(x)
x

"inside double" = "inside double"
x = 11
x = 22
double(x) = 22


11

Note that we have updated variables in many functions, like `newton` without consequences and that is fine as long as non objects (strings, numbers) are updated.  However, if we have an array, we can update the value of this.  Consider

In [48]:
function doubleThird!(arr::Vector{T}) where T <: Real
  arr[3] *= 2
  arr
end

doubleThird! (generic function with 1 method)

In [49]:
x = collect(1:5)
doubleThird!(x)
x

5-element Vector{Int64}:
 1
 2
 6
 4
 5

What's going on here?  

And remember that if we are updating a mutable argument inside of a function, then we should add a `!` to the end of the name of the function.  Again, this is a Julia convention only, but good to do. 

The main type of argument that this works with is an array, but we will see other objects (`Dict`s and `DataFrame`s) later.

### Summary

- all arguments should be checked if it is important that they fall within a range. 
- one can use optional arugments (that have default values) instead of fixed values within a function.
- also, one can use keyword arguments (in the form `opt = value`) that have default values. 
- Parametric methods are helpful to keep the code from being copied for different data types.
- understanding how function arguments are handled within a function.